In [2]:
import numpy as np
import pandas as pd
import kagglehub
from PIL import Image
import os
from itertools import islice
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from torchvision.io import decode_image
from torch.utils.data import Dataset
from Image_Processing import CustomImageDataset, find_jpg_files, generate_dataframe
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

path = kagglehub.dataset_download("fahadullaha/facial-emotion-recognition-dataset")
SEARCH_ROOT_DIR = f"{path}/processed_data/"
# fpath = find_jpg_files(SEARCH_ROOT_DIR)
# df = generate_dataframe(fpath)

In [ ]:
# Reload the Image_Processing module to get the latest changes
import importlib
import Image_Processing
importlib.reload(Image_Processing)
from Image_Processing import CustomImageDataset, find_jpg_files, generate_dataframe

In [3]:
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, )
# # Save the split dataframes to CSV files for CustomImageDataset
# train_df.to_csv("train_annotations.csv", index=False)
# test_df.to_csv("test_annotations.csv", index=False)

# Create Dataset objects using your CustomImageDataset class
train_dataset = CustomImageDataset("train_annotations.csv", SEARCH_ROOT_DIR)
test_dataset = CustomImageDataset("test_annotations.csv", SEARCH_ROOT_DIR)

# Create DataLoaders from the Dataset objects
train_loader = torch.utils.data.DataLoader(
    train_dataset,  # Use the Dataset object, not the DataFrame
    batch_size=64,  # number of samples loaded per batch
    shuffle=True,   # randomize the order of samples
    num_workers=2   # number of processes used to load data
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,   # Use the Dataset object, not the DataFrame
    batch_size=64,
    shuffle=False,
    num_workers=2
)

In [4]:
import torch.optim as optim
resnet_model = models.resnet18(weights=None)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 8)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001) # lr: learning rate affects how quickly the model adjusts parameters
     

In [5]:
from tqdm import tqdm

running_loss = 0.0
resnet_model.train()  # set model to training mode

# Initialize as empty list (more efficient than np.append in a loop)
batch_nums = []
loss_values = []

# Wrap the train_loader with tqdm for a progress bar
for i, (images, labels) in enumerate(tqdm(train_loader, desc="Training", unit="batch")): 
    outputs = resnet_model(images)        # get model predictions for this batch of images
    loss = loss_function(outputs, labels) # calculate loss

    # Track running loss on training data
    running_loss += loss.item()
    
    # Store batch number and loss value
    batch_nums.append(i)
    loss_values.append(loss.item())
    
    loss.backward()                       # compute gradients for each weight (how each weight should change to minimize loss)
    optimizer.step()                      # update model weights
    
    optimizer.zero_grad()                 # reset gradients from the previous batch

print(f"Average training loss: {running_loss / len(train_loader):.4f}")
print("Finished Training")

# Plot the training loss over batches
plt.plot(batch_nums, loss_values, 'o-', alpha=0.6)
plt.xlabel('Batch Number')
plt.ylabel('Loss')
plt.title('Training Loss per Batch')
plt.grid(True, alpha=0.3)
plt.show()

Training:  13%|█▎        | 81/623 [00:30<03:24,  2.65batch/s]


KeyboardInterrupt: 

In [ ]:
correct, total = 0, 0
resnet_model.eval()    # set the model to evaluation mode
with torch.no_grad():  # disable gradient tracking since we’re not updating weights
    for images, labels in test_loader: 
        outputs = resnet_model(images)                 # get model predictions for a batch of images
        _, predicted = torch.max(outputs, 1)           # select the class with the highest score
        total += labels.size(0)                        # count total number of images processed
        correct += (predicted == labels).sum().item()  # count how many predictions were correct

print(f"Test accuracy: {100 * correct / total:.2f}%")

In [ ]:

torch.save(resnet_model, 'model.pth') # save model to model.pth file